In [1]:
import math

import numpy
import scipy


def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    d1 = (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (
            volitility * math.sqrt(mature_time))
    d2 = (d1 - (volitility * math.sqrt(mature_time)))
    call = (spot_price * scipy.stats.norm.cdf(d1)) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf(d2)
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -d2) - spot_price * scipy.stats.norm.cdf(-d1)
    if callput == "call":
        return call
    else:
        return put
    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))

In [2]:
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))
# black_scholes(spot_price,strike_price,risk_rate,mature_time,volatility)

1.0226152225558796
18.05980466595578


In [3]:
import math
import numpy
import scipy


def binomial(starting_value, strike, rate, volitility, time, step, type, excersise):
    up = math.exp(volitility * math.sqrt(time / step))
    down = 1 / math.exp(volitility * math.sqrt(time / step))
    rate_value = math.exp((-rate) * time / step)
    increase_probability = ((math.exp(rate * (time / step)) - down) / (up - down))
    decrease_probability = 1 - ((math.exp(rate * (time / step)) - down) / (up - down))
    tree = [[0]]
    tree[0][0] = tree[0][0] + starting_value
    for cur_depth in range(1, step + 1):
        new_values = []
        old = tree[-1]
        for child in range(0, cur_depth + 1):
            if cur_depth == child:
                new = old[-1] * down
            elif child == 0:
                new = old[0] * up
            else:
                new = ((old[child - 1] * up) + (old[child] * down)) / 2
            new_values.append(new)
        tree.append(new_values)
    children = tree[-1]
    values = []
    for i in children:
        if type == "put":
            if strike - i > 0:
                option = strike - i
            else:
                option = 0
        else:
            if i - strike > 0:
                option = i - strike
            else:
                option = 0
        values.append(option)
    i = step - 1
    while i >= 0:
        for j in range(i + 1):
            values[j] = (increase_probability * values[j + 1] + decrease_probability * values[j]) * rate_value
        i = i - 1
    print(tree)
    print(values)
    return values[0]


starting_value = 100
strike = 110
rate = 0.05
volatility = 0.2
time = 1
step = 100
type = 'call'
exercise = 'European'
#binomial(100,110,0.05,0.2,1,100,'call','euro')
# binomial(20,21,0.12,0.1,0.5,2,"call","euro")
binomial(50, 52, 0.05, 0.2, 2, 2, "put", "e")

[[50], [61.07013790800849, 40.936537653899094], [74.59123488206352, 54.05361859192274, 33.51600230178197]]
[5.577764856335021, 10.153787114456259, 18.483997698218033]


5.577764856335021

In [4]:
import numpy


def monte_carlo(stock_price, interest_rate, volitility, time, simulations):
    value = math.log(stock_price)
    for i in range(0, simulations):
        value = value + (interest_rate - 1 / 2 * (volitility * volitility)) * time / simulations * math.sqrt(
            time / simulations) * numpy.random.normal(size=1) + volitility * numpy.random.normal(size=1)
    return math.exp(value)


monte_carlo(1, 2, 3, 4, 5)

192095.9270170842